In [3]:
import boto3
import random
import dotenv
import json
#load env from .env
dotenv.load_dotenv()
bedrock_runtime = boto3.client('bedrock-runtime')

In [2]:

from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [8]:
import json
with open('/Users/chuanxie/Downloads/trans_right_result_v2.json','r') as f:
    data = json.load(f)

In [18]:
data.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28'])

In [31]:
data['1']

{'en-US': ['快乐医院', 'Happy Hospital', '10'],
 'ja-JP': ['Happy Hospital', 'ハッピーホスピタル', '8'],
 'de-DE': ['Happy Hospital', 'Glückliches Krankenhaus', '8'],
 'es-ES': ['Happy Hospital', 'Hospital Feliz', '10'],
 'vi-VN': ['Happy Hospital', 'Bệnh viện Hạnh phúc', '9'],
 'th-TH': ['Happy Hospital', 'โรงพยาบาลที่มีความสุข', '10'],
 'it-IT': ['Happy Hospital', 'Ospedale Felice', '10'],
 'pt-PT': ['Happy Hospital', 'Hospital Feliz', '10'],
 'id-ID': ['Happy Hospital', 'Rumah Sakit Bahagia', '9'],
 'fr-FR': ['Happy Hospital', 'Hôpital Heureux', '8'],
 'ko-KR': ['Happy Hospital', '행복한 병원', '9'],
 'zh-HK': ['Happy Hospital', '快樂醫院', '10']}

In [21]:
prompt_trans = \
"""
You are an expert translator that will be tasked with translating a piece of text. 
Please translate the {source_lang} language to {target_lang} language using the following rules:
Output only the translation result without explanation.
Do not self reference.
Ensure that the meaning of the original text is not changed.
Try to translate all content, not piece of it. 5.Keep the original flavor/taste.

Here is the original text:
<text>
{text}
</text>

Skip the preamble, go straight into the answer.
"""

In [63]:
prompt_eval = \
"""
You are a Translation scorer, give you a translated text from language {source_lang} to language {targe_lang}. 
you must to give a score from 0 - 10, 0 being the worst and 10 being the best.

For example :

<translation>
{{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'en-US', 'translated': 'Happy Hospital'}}
</translation>
Your output must be a score enclosed by xml tag <score>:
<score>9.5<score>

Here is the translation:
<translation>
{translation}
</translation>

Skip the preamble, go straight into the answer.
"""

In [64]:
prompt_template_trans = ChatPromptTemplate.from_template(prompt_trans)
prompt_template_eval = ChatPromptTemplate.from_template(prompt_eval)

In [23]:

llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.2,
                                "top_k":250,
                                "max_tokens": 1024,
                                "top_p":0.95,
                                # "stop_sequences":['</response>']
                               },
                  streaming=False,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0.2,
                                "top_k":250,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=False,callbacks=[StreamingStdOutCallbackHandler()])

llm_gpt= ChatOpenAI( model='gpt-3.5-turbo',
                     temperature= 0.2,
                    )

llm_gpt4= ChatOpenAI( model='gpt-4-0125-preview',
                     temperature= 0.2,
                    )

In [45]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain_haiku = prompt_template_trans | llm_haiku |output_parser
chain_sonnet = prompt_template_trans | llm_sonnet |output_parser

In [51]:
values = [item[1] for item in list(data.items())]
values

[{'en-US': ['快乐医院', 'Happy Hospital', '10'],
  'ja-JP': ['Happy Hospital', 'ハッピーホスピタル', '8'],
  'de-DE': ['Happy Hospital', 'Glückliches Krankenhaus', '8'],
  'es-ES': ['Happy Hospital', 'Hospital Feliz', '10'],
  'vi-VN': ['Happy Hospital', 'Bệnh viện Hạnh phúc', '9'],
  'th-TH': ['Happy Hospital', 'โรงพยาบาลที่มีความสุข', '10'],
  'it-IT': ['Happy Hospital', 'Ospedale Felice', '10'],
  'pt-PT': ['Happy Hospital', 'Hospital Feliz', '10'],
  'id-ID': ['Happy Hospital', 'Rumah Sakit Bahagia', '9'],
  'fr-FR': ['Happy Hospital', 'Hôpital Heureux', '8'],
  'ko-KR': ['Happy Hospital', '행복한 병원', '9'],
  'zh-HK': ['Happy Hospital', '快樂醫院', '10']},
 {'en-US': ['医生等级越高，他们治疗病人就会越快。',
   "The higher the doctor's level, the faster they can treat patients.",
   '9'],
  'ja-JP': ["The higher the doctor's level, the faster they can treat patients.",
   '医者のレベルが高いほど、患者の治療が速くなります。',
   '9'],
  'de-DE': ["The higher the doctor's level, the faster they can treat patients.",
   'Je höher das Niveau des A

In [60]:
samples = []
for item in values:
    source_lang = 'zh-CN'
    text_zh = item['en-US'][0]
    for key in list(item.keys()):
        samples.append({'source_lang':source_lang,'text':text_zh,'target_lang':key})

In [43]:
source_lang = 'zh-CN'
target_lang = 'en-US'
target_lang = 'vi-VN'

text = """疾病不止，医者不息。"""
resp = chain_haiku.invoke({'source_lang':source_lang,"target_lang":target_lang,"text":text})
print(resp)

Bệnh tật không ngừng, người thầy thuốc không nghỉ.


In [62]:
results = []
for sample in samples:
    resp = chain_haiku.invoke(
            {'source_lang':sample['source_lang'],"target_lang":sample['target_lang'],"text":sample['text']}
            )
    results.append({**sample,"translated":resp})
    print({**sample,"translated":resp})



{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'en-US', 'translated': 'Happy Hospital'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'ja-JP', 'translated': '幸せな病院'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'de-DE', 'translated': 'Fröhliches Krankenhaus'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'es-ES', 'translated': 'Hospital de la alegría'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'vi-VN', 'translated': 'Bệnh viện Vui Vẻ'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'th-TH', 'translated': 'โรงพยาบาลแห่งความสุข'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'it-IT', 'translated': 'Ospedale felice'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'pt-PT', 'translated': 'Feliz Hospital'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'id-ID', 'translated': 'Rumah Sakit Bahagia'}
{'source_lang': 'zh-CN', 'text': '快乐医院', 'target_lang': 'fr-FR', 'translated': 'Hôpital joyeux'}
{'source_

KeyboardInterrupt: 

In [46]:
resp = chain_sonnet.invoke({'source_lang':source_lang,"target_lang":target_lang,"text":text})
print(resp)

Bệnh tật không ngừng, người thầy thuốc không nghỉ.
